In [4]:
# 1. 데이터 파일 읽기
import pandas as pd

# 로컬 JSON 파일 읽기
df = pd.read_csv("./data/train_dataset.csv", encoding='utf-8')
df.head()

,type,original_target_part,golden_target_part,err_sentence,cor_sentence
0,조사오류,앉은 모습은 처음으로 언론,앉은 모습이 처음으로 언론,이날 재판에선 박 전 대통령이 피고인석에 앉은 모습은 처음으로 언론에 공개됐다. 재...,이날 재판에선 박 전 대통령이 피고인석에 앉은 모습이 처음으로 언론에 공개됐다. 재...
1,맞춤법,금새,금세,하늘에 먹구름이 몰려오더니 금새 비원 앞 광장에 비가 쏟아지기 시작했다. 기상청 아...,하늘에 먹구름이 몰려오더니 금세 비원 앞 광장에 비가 쏟아지기 시작했다. 기상청 아...
2,비문,영구히' 중국을 다시 방문할 불허하겠다,중국 재방문을 '영구히' 불허하겠다,불상사를 예방하려면 중국 방문시 발급 받은 비자가 허가하는 체류 기간을 확인하고 입...,불상사를 예방하려면 중국 방문시 발급 받은 비자가 허가하는 체류 기간을 확인하고 입...
3,조사오류,재팬 쇼크는 유럽 경제에 끼친 파장은,재팬 쇼크가 유럽 경제에 끼친 파장은,-재팬 쇼크는 유럽 경제에 끼친 파장은.,-재팬 쇼크가 유럽 경제에 끼친 파장은.
4,사이시옷,고개짓,고갯짓,"무대 위엔 으스스한 분위기의 문 하나, 작은 의자와 배우 한 명뿐. 처음 자신을 브...","무대 위엔 으스스한 분위기의 문 하나, 작은 의자와 배우 한 명뿐. 처음 자신을 브..."


In [5]:
# 2. 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   type                  254 non-null    object
 1   original_target_part  254 non-null    object
 2   golden_target_part    254 non-null    object
 3   err_sentence          254 non-null    object
 4   cor_sentence          254 non-null    object
dtypes: object(5)
memory usage: 10.1+ KB


In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage
from langchain_ollama import ChatOllama


def process(text, examples):

    llm = ChatUpstage(model="solar-pro2", temperature=0.0)
    #llm = ChatOllama(model="alibayram/Qwen3-30B-A3B-Instruct-2507")
    #llm = ChatOllama(model="gpt-oss:120b-cloud", temperature=0.1)
    
    sample_section = "\n".join(
        f"<원문>\n{src}\n<교정>\n[Answer]:{tgt}\n" for src, tgt in examples
    )


    #print(sample_section)

    # 프롬프트 
    selectCategory = """
    # 지시
    - 당신은 한국어 문장 교정 전문가입니다. 
    - 다음 규칙에 따라 원문을 교정하세요.
    - 맞춤법, 띄어쓰기, 문장 부호, 문법을 자연스럽게 교정합니다.
    - 반드시 문법적으로 교정해야만 하는 원문만 교정합니다.
    - 불필요한 내용이어도 문법적으로 이상이 없다면 교정하지 않습니다.
    - 문장부호는 확실하게 틀렸을 경우에만 교정합니다.
  
    # 예시
    {sample_section}

    # 교정할 문장
    <원문>
    {message}
    <교정>
    [Answer]:
    """

    # 프롬프트 객체 생성
    prompt = PromptTemplate(
        input_variables=["message"],
        template=selectCategory
    )

    # 출력 파서 (문자열)
    output_parser = StrOutputParser()

    # LCEL 체인 구성
    chain = (
        prompt 
        | llm 
        | output_parser
    )

    
    result = chain.invoke({"message": text, "sample_section": sample_section})

    return result

In [9]:
# 6. 메인
import re
import importlib
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from mecab import MeCab

mecab = MeCab()

def get_top_n_structure_similar(df, target_sentence, n=10):
    # "msg" 컬럼(문장 리스트) → 품사 시퀀스 변환
    
    err_sentence_list = df["err_sentence"].tolist()
    sentence_list = df["cor_sentence"].tolist()
    sentence_list_pos = [" ".join([tag for _, tag in mecab.pos(s)]) for s in sentence_list]
    target_pos = " ".join([tag for _, tag in mecab.pos(target_sentence)])
    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform([target_pos] + sentence_list_pos)
    sims = cosine_similarity(X[0], X[1:])[0]
    
    top_idxs = sims.argsort()[::-1][:n]
    # 결과를 (원문, 유사도) 튜플로 반환
    #return [err_sentence_list[idx] for idx in top_idxs], [(sentence_list[idx], sims[idx]) for idx in top_idxs]
    return list(zip([err_sentence_list[idx] for idx in top_idxs], [sentence_list[idx] for idx in top_idxs]))

def extract_answer(text):
    # [Answer]: 뒤에 오는 줄의 맨 앞~줄 끝까지 추출
    match = re.search(r"\[Answer\]:\s*(.*)", text)
    if match:
        return match.group(1).strip()
    return ""


def main():

    errorType = []
    for idx, row in df.iterrows():

        if idx == 10: break
        examples = get_top_n_structure_similar(df, row['cor_sentence'])
        print(examples)

        result = process(row['err_sentence'], examples)

        print(row['err_sentence'])
        print("-"*25)
        #print(extract_answer(result))
        print(result)
        print("*"*25)
        errorType.append(result)

    #df['new'] = errorType
    #df.to_csv('./new.csv', index=False)    


main()



[('이날 재판에선 박 전 대통령이 피고인석에 앉은 모습은 처음으로 언론에 공개됐다. 재판부가 사진과 영상 촬영을 허가했기 때문이다. 박 전 대통령은 재판에서 8분 동안 발언을 하며 "계엄령은 그 자체로 가치 중립적인 법적 수단에 불과하다" "계엄과 내란은 다르다" 등의 주장을 펼쳤다.', '이날 재판에선 박 전 대통령이 피고인석에 앉은 모습이 처음으로 언론에 공개됐다. 재판부가 사진과 영상 촬영을 허가했기 때문이다. 박 전 대통령은 재판에서 8분 동안 발언을 하며 "계엄령은 그 자체로 가치 중립적인 법적 수단에 불과하다" "계엄과 내란은 다르다" 등의 주장을 펼쳤다.'), ('삼성전자의 공장은 사람의 개입을 최소화했다. FC-BGA 같은 고난도 초미세 공정을 요하는 반도체 기판 제품의 경우 아주 조그마한 이물질도 품질 불량으로 이어질 수 있기 때문이다. 이날 생산 공장 곳곳에서는 설비들 사이로 자동로봇(AMR) 수십 대가 쉴 새 없이 오가며 자재를 운반하고 있었다. 입력된 고객 납기 기간에 맞춰 로봇에 자동으로 명령이 내려지고, 이에 따라 원자재를 각 공정에 맞는 설비로 운반한다. 장비 유지와 보수 인력 이외에 10여 단계의 공정을 모두 무인화했다. 삼성전자는 "현재 기존 공장 대비 50% 수준의 인원으로 운영되고 있으며, 최종적으로는 완전 무인화 수준으로 운영하는 것이 목표"라고 했다.', '삼성전자의 공장은 사람의 개입을 최소화했다. FC-BGA 같은 고난도 초미세 공정을 요하는 반도체 기판 제품의 경우 아주 조그마한 이물질도 품질 불량으로 이어질 수 있기 때문이다. 이날 생산 공장 곳곳에서는 설비들 사이로 자동로봇(AMR) 수십 대가 쉴 새 없이 오가며 자재를 운반하고 있었다. 입력된 고객 납기에 맞춰 로봇에 자동으로 명령이 내려지고, 이에 따라 원자재를 각 공정에 맞는 설비로 운반한다. 장비 유지와 보수 인력 이외에 10여 단계의 공정을 모두 무인화했다. 삼성전자는 "현재 기존 공장 대비 50% 수준의 인원으로 운영되고 있으며, 최종적으로는 완전 무인